## Desafio de Análise de Dados: Empréstimos de Acervos das Bibliotecas da UFRN

Neste desafio do #7DaysOfCode, o objetivo é analisar os dados de empréstimos dos acervos do sistema de bibliotecas da UFRN, utilizando as ferramentas do Python como o Pandas. A partir dessa análise, será possível extrair insights valiosos sobre o comportamento dos empréstimos ao longo dos últimos anos, identificar padrões e formular hipóteses para melhorar a gestão e os processos das bibliotecas.

A análise exploratória de dados (EDA) será a principal metodologia utilizada. A EDA é fundamental para compreender as tendências nos dados, explorar informações sobre os processos, usuários e recursos das bibliotecas, além de gerar novas perguntas que podem orientar decisões mais assertivas na gestão do acervo. Durante o processo, vamos focar em questões como:

A quantidade de empréstimos tem aumentado ou diminuído ao longo dos anos?

Quais bibliotecas apresentam o maior número de empréstimos?

Quais são os temas mais e menos requisitados pelos usuários?

Para atingir esses objetivos, o trabalho será dividido em várias etapas, desde a coleta e transformação dos dados até a criação de visualizações e a exportação de tabelas para um formato utilizável em diferentes contextos. O Pandas será utilizado para importar dados de diferentes fontes, realizar transformações necessárias e, por fim, gerar insights que possam ser apresentados de forma clara e eficaz.

Com esses dados, será possível entender melhor o funcionamento do sistema de bibliotecas e fornecer informações para tomadas de decisão mais informadas, visando melhorar a experiência dos usuários e a utilização dos recursos disponíveis.


In [ ]:
# Instalação das bibliotecas necessárias
%pip install pandas
%pip install fastparquet
%pip install seaborn
%pip install matplotlib
%pip install plotly
%pip install numpy


In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
from io import StringIO

### Coletando os dados de empréstimos (2010 a 2020)

In [ ]:
# Caminho para a pasta com os arquivos
caminho = 'dataset/dados_emprestimos/*.csv'

# Obter todos os arquivos CSV
arquivos = glob.glob(caminho)

# Lista para armazenar os DataFrames
dfs = []

# Ler cada arquivo e adicionar à lista
for arquivo in arquivos:
    df = pd.read_csv(arquivo)
    dfs.append(df)

print(f"Total de arquivos lidos: {len(arquivos)}")

### Concatenando os DataFrames

In [ ]:
emprestimos = pd.concat(dfs, ignore_index=True)
emprestimos.value_counts()

### Verificando Duplicatas

In [ ]:
duplicatas = emprestimos[emprestimos.duplicated()]
print(f"Total de duplicatas encontradas: {len(duplicatas)}")

### Excluindo Duplicatas

In [ ]:
emprestimos = emprestimos.drop_duplicates()

In [ ]:
# Exibindo as primeiras linhas do DataFrame
emprestimos

### Importando os Dados de exemplares

In [ ]:
dados_exemplares = pd.read_parquet('https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_1-Importando_dados/Datasets/dados_exemplares.parquet')
dados_exemplares

In [ ]:
todos_emprestimos = emprestimos.merge(dados_exemplares, on="codigo_barras", how="inner")
todos_emprestimos.head()

### Verificando a estrutura do DataFrame

In [ ]:
todos_emprestimos.dtypes

### Criando a coluna "CDU" com base na classificação da localização


In [ ]:
def classificar_CDU(valor):
    if valor < 100:
        return 'Generalidades, Ciência e Conhecimento'
    elif valor < 200:
        return 'Filosofia e Psicologia'
    elif valor < 300:
        return 'Religião'
    elif valor < 400:
        return 'Ciências sociais'
    elif valor < 500:
        return 'Classe vaga'
    elif valor < 600:
        return 'Matemática e Ciências Naturais'
    elif valor < 700:
        return 'Ciências aplicadas'
    elif valor < 800:
        return 'Belas Artes'
    elif valor < 900:
        return 'Linguagem'
    else:
        return 'Geografia, Biografia e História'

todos_emprestimos['CDU'] = todos_emprestimos['localizacao'].apply(classificar_CDU)
todos_emprestimos.head()


### Excluindo a coluna "registro_sistema" do DataFrame

In [ ]:
todos_emprestimos.drop('registro_sistema', axis=1, inplace=True)
todos_emprestimos.columns

### Transformando a coluna "matricula_ou_siape" para o tipo string

In [ ]:
todos_emprestimos['matricula_ou_siape'] = todos_emprestimos['matricula_ou_siape'].astype(str)
todos_emprestimos['matricula_ou_siape'].dtypes

### Relação entre empréstimo e empréstimo de exemplar

In [ ]:
todos_emprestimos['id_emprestimo'].value_counts()

### Total de empréstimos e empréstimo de exemplares

In [ ]:
qtd_emprestimos = todos_emprestimos['id_emprestimo'].nunique()
print(f"Quantidade total de empréstimos: {qtd_emprestimos}")
qtd_exemplares = len(todos_emprestimos)
print(f"Quantidade total de exemplares: {qtd_exemplares}")

### Total de exemplares emprestados por ano

In [ ]:
data_emprestimos = pd.DataFrame(todos_emprestimos['data_emprestimo'].value_counts()).reset_index()
data_emprestimos.columns = ['data','quantidade']
data_emprestimos['data'] = pd.to_datetime(data_emprestimos['data'])
data_emprestimos

### Funções para agrupar por período e plotar o gráfico

In [ ]:
def agrupar_por_periodo(df, coluna_data, coluna_valor, periodo):
    """
    Agrupa os dados por um período específico (mês, ano ou hora) e retorna a soma dos valores.
    :param df: DataFrame com os dados.
    :param coluna_data: Nome da coluna com as datas.
    :param coluna_valor: Nome da coluna com os valores a serem somados.
    :param periodo: Período para agrupar ('month', 'year' ou 'hour').
    :return: DataFrame agrupado com a soma dos valores.
    """

    if not pd.api.types.is_datetime64_any_dtype(df[coluna_data]):
        df = df.copy()
        df[coluna_data] = pd.to_datetime(df[coluna_data])
    
    if periodo == 'month':
        resultado = df.groupby(df[coluna_data].dt.month)[coluna_valor].sum()
        meses = {1:'Jan', 2:'Fev', 3:'Mar', 4:'Abr', 5:'Mai', 6:'Jun', 
                7:'Jul', 8:'Ago', 9:'Set', 10:'Out', 11:'Nov', 12:'Dez'}
        resultado.index = resultado.index.map(meses)
        resultado.index.name = 'mês'
    elif periodo == 'year':
        resultado = df.groupby(df[coluna_data].dt.year)[coluna_valor].sum()
        resultado.index.name = 'anos'
    elif periodo == 'hour':
        resultado = df.groupby(df[coluna_data].dt.hour)[coluna_valor].sum()
        resultado.index = [f"{h:02d}h" for h in resultado.index]
        resultado.index.name = 'hora'
    
    return resultado


def plotar_grafico(dados, titulo, tipo='line'):
    """
    Plota um gráfico com os dados fornecidos.
    :param dados: Dados a serem plotados (DataFrame ou Series).
    :param titulo: Título do gráfico.
    :param tipo: Tipo de gráfico ('line' ou 'bar').
    :return: Objeto do gráfico.
    """
    sns.set_theme(style="darkgrid")
    fig, ax = plt.subplots(figsize=(12, 6))
    
    if tipo == 'line':
        sns.lineplot(x=dados.index, y=dados.values, marker='o', linewidth=2, color='#1f77b4', ax=ax)          
    else:
        sns.barplot(x=dados.index, y=dados.values, hue=dados.index, palette='colorblind', ax=ax, legend=False)

    
    if dados.index.name == 'anos':
        ax.set_xticks(dados.index)
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_title(titulo, fontsize=14, fontweight='bold')
        
    elif dados.index.name == 'mês':
        ax.set_xticks(range(len(dados.index)))
        ax.set_xticklabels(dados.index, rotation=45, ha='right')
        ax.set_title(titulo, fontsize=14, fontweight='bold')

    else:
        ax.set_xticks(range(len(dados.index)))
        ax.set_title(titulo, fontsize=16, fontweight='bold')
        ax.set_xlabel('')
        ax.set_ylabel('')

    plt.tight_layout()
    return fig, ax        


In [ ]:
agrupar_por_periodo(todos_emprestimos, 'data_emprestimo', 'id_emprestimo', 'month').head(10)

### Análise por ano

In [ ]:
emprestimo_ano = agrupar_por_periodo(data_emprestimos, 'data', 'quantidade', 'year')
fig_ano, ax_ano = plotar_grafico(emprestimo_ano, 'Empréstimos por Ano', tipo='line')


A análise mostra que desde 2013 o número de exemplares emprestados vem diminuindo de forma contínua, com uma queda significativa já em 2019. O gráfico mostra que, após o pico de mais de 230 mil empréstimos em 2013, houve uma leve redução em 2014, certa estabilidade até 2017, e então uma queda mais acentuada em 2018 e 2019. Já o ano de 2020 apresentou uma quantidade de empréstimos muito inferior aos anos anteriores, mas não pode ser avaliado da mesma forma devido à diferença no volume de dados disponíveis para esse período.

É importante que a direção da biblioteca investigue com mais profundidade o que está ocorrendo desde 2013, revisando os procedimentos e buscando entender os fatores que contribuíram para essa redução. Algumas perguntas importantes que podem orientar análises futuras:

* O número de alunos da universidade vem diminuindo?

* A biblioteca está aplicando alguma pesquisa de satisfação com os usuários? Como estão os resultados?

* O acervo está atualizado e atrativo para os estudantes e pesquisadores?

* Quais estratégias de divulgação e marketing estão sendo realizadas para incentivar o uso da biblioteca?

* O que ocorreu de diferente em 2017 em comparação com 2015 e 2016, que pode ter contribuído para uma leve recuperação nos números?

*E em 2019? Houve alguma mudança estrutural, de política ou de atendimento?

### Análise por mês

In [ ]:
emprestimo_mes = agrupar_por_periodo(data_emprestimos , 'data', 'quantidade', 'month')
fig_mes, ax_mes = plotar_grafico(emprestimo_mes, 'Empréstimos por Mês', tipo='line')

Nota-se que os maiores volumes de empréstimos acontecem nos meses de março e agosto, o que faz bastante sentido considerando que são meses de início de semestre nas universidades.

Já os meses de janeiro, julho e dezembro apresentam os menores números, o que provavelmente está relacionado às férias acadêmicas. Isso não surpreende, mas é importante considerar que mesmo nesses períodos o funcionamento da biblioteca precisa ser garantido. Por isso, é essencial pensar no revezamento das equipes, evitando que todos tirem férias ao mesmo tempo, especialmente em janeiro, quando podem ser feitas atividades como inventário do acervo ou manutenção geral.

Os meses de maio, junho, outubro e novembro, que têm movimentação intermediária, podem ser bem aproveitados pela gestão para treinamentos, eventos internos ou pequenas reformas, sem impactar tanto o fluxo de usuários.

Chama atenção a queda expressiva em junho, o que merece uma investigação mais aprofundada. Pode haver algum fator pontual interferindo, como greves, calendário acadêmico alterado ou mesmo eventos que diminuem o interesse pelo uso da biblioteca nesse período.

Uma boa sugestão seria planejar ações específicas de incentivo ao uso da biblioteca antes das férias, como campanhas para empréstimo de livros para leitura em casa, desafios de leitura ou até divulgação especial nas redes sociais.

Essas ações podem ajudar a equilibrar o fluxo ao longo do ano e manter a biblioteca sempre presente na rotina dos estudantes.

### Análise por hora

In [ ]:
emprestimo_hora = agrupar_por_periodo(data_emprestimos, 'data', 'quantidade', 'hour')
fig_hora, ax_hora = plotar_grafico(emprestimo_hora, 'Empréstimos por Hora', tipo='bar')

Baseado no gráfico, vejo que há um pico significativo no atendimento de empréstimos durante as 10 h e 16 h, que são os horários de maior demanda. Isso indica que esses períodos podem exigir mais pessoal ou recursos para lidar com a demanda intensa. Além disso, pode ser interessante focar na melhoria do processo de atendimento nesses horários de pico para otimizar a eficiência.

Por outro lado, os períodos mais tranquilos, entre 7h e 9h da manhã e 19h a 22h, são ideais para realizar atividades que não dependem tanto da interação direta com os clientes, como atualização de registros, organização dos empréstimos ou planejamento das próximas etapas. Aproveitar esses horários de menor movimento pode ajudar a manter o fluxo de trabalho equilibrado ao longo do dia e garantir que as operações se mantenham eficientes.

### Analisando variáveis categóricas

Agora vamos analisar as variáveis categóricas do conjunto de dados de empréstimos da biblioteca. A ideia é entender melhor o comportamento dos usuários a partir de categorias como tipo de vínculo, coleção, biblioteca e classificação geral da CDU.

Criei uma função para gerar tabelas de frequência e percentual dessas variáveis, facilitando a visualização dos dados e a comparação entre elas. Com base nos resultados, surgem insights importantes para a diretoria: como se distribuem os empréstimos entre os tipos de vínculo, quais coleções são mais procuradas, quais bibliotecas têm mais movimentação e quais temas da CDU são mais emprestados.

Essas análises ajudam a direcionar ações, avaliar estratégias atuais e identificar oportunidades de melhoria na gestão do acervo e no atendimento ao público.

### Verificando os valores unicos 

In [ ]:
todos_emprestimos.head()

In [ ]:
# Valores únicos para tipo de vínculo do usuário
print("TIPO_VINCULO_USUARIO")
print(todos_emprestimos.tipo_vinculo_usuario.unique())
print("Total:", len(todos_emprestimos.tipo_vinculo_usuario.unique()))

# Valores únicos para coleção
print("\nCOLECAO")
print(todos_emprestimos.colecao.unique())
print("Total:", len(todos_emprestimos.colecao.unique()))

# Valores únicos para biblioteca
print("\nBIBLIOTECA")
print(todos_emprestimos.biblioteca.unique())
print("Total:", len(todos_emprestimos.biblioteca.unique()))

# Valores únicos para CDU
print("\nCDU")
print(todos_emprestimos.CDU.unique())
print("Total:", len(todos_emprestimos.CDU.unique()))

### Função para gerar tabela de contagem de frequência

In [ ]:
def tabela_frequencia(df, variavel):
  '''
  Esta função gera uma tabela de frequência com percentuais de acordo 
  com a variável categórica passada.
  :param df: DataFrame com os dados.
  :param variavel: Nome da variável categórica a ser analisada.
  :return: DataFrame com a tabela de frequência e percentuais.
  
  '''


  dataframe = pd.DataFrame(df[variavel].value_counts())                      
  dataframe.columns = ['quantidade']
  dataframe['percentual'] = round((dataframe.quantidade / dataframe.quantidade.sum())*100, 1)

  return dataframe

### Frequência por tipo de vinculo do usuário

In [ ]:
tabela_frequencia(todos_emprestimos, 'tipo_vinculo_usuario')

Com base na distribuição dos empréstimos por tipo de vínculo, podemos perceber que o aluno de graduação é, de longe, o principal usuário da biblioteca, representando 77,8% do total de empréstimos. Isso mostra que a biblioteca está muito conectada com a rotina dos cursos de graduação, o que é positivo, mas também indica onde está a maior demanda e, portanto, onde o serviço precisa ser mais robusto. 
Os alunos de pós-graduação aparecem em segundo lugar com 14,6%, o que reforça a importância de manter acervos atualizados e especializados para esse público.

### Frequência por coleção

In [ ]:
tabela_frequencia(todos_emprestimos, 'colecao')

Ao analisar a distribuição dos empréstimos por tipo de coleção, fica evidente que o Acervo Circulante domina amplamente, representando 99,1% de todos os empréstimos. Isso mostra que ele é o principal recurso utilizado pelos usuários, o que levanta um alerta sobre o baixo uso das demais coleções.

Coleções relevantes para a pesquisa acadêmica, como monografias (0,2%), dissertações (0,1%) e teses (0,0%), têm participação mínima nos empréstimos. Isso pode indicar um potencial pouco explorado para o fortalecimento da cultura científica e investigativa na universidade. No entanto, é importante verificar se esses materiais estão disponíveis online, o que justificaria o baixo número de empréstimos físicos.

Além disso, coleções especiais como a Coleção Mossoroense, Coleção Zila Mamede, Literatura de Cordel e Obras Raras apresentam números extremamente baixos de empréstimos. Essas coleções, por seu valor histórico, cultural ou regional, merecem atenção especial. É preciso entender se a baixa procura se deve à falta de divulgação, restrições de acesso ou desconhecimento do acervo.

### Frequência por blibliotecas

In [ ]:
tabela_frequencia(todos_emprestimos, 'biblioteca')

Como já era esperado, a Biblioteca Central Zila Mamede se destaca com folga, representando quase 69% de todos os empréstimos do sistema. Isso reflete seu porte, localização e acervo mais diversificado, sendo o principal ponto de apoio da comunidade acadêmica.

Apesar disso, é interessante ver que outras bibliotecas setoriais também têm participação significativa, como a do CCS, FACISA (Santa Cruz) e o Departamento de Odontologia, o que mostra que essas unidades têm papel relevante dentro dos seus contextos específicos.

Esse ranking é útil para visualizar o panorama geral do sistema de bibliotecas, mas só os números absolutos não contam toda a história. Para uma análise mais justa e completa, o ideal seria cruzar com outras métricas, como:

- Quantos empréstimos por usuário cadastrado cada biblioteca realiza

- Qual seria o volume esperado de empréstimos, considerando o público atendido

- Quais ações e investimentos têm sido feitos em cada unidade

Essas comparações ajudam a entender melhor onde o sistema está funcionando bem, onde há espaço para melhorias e como distribuir melhor os recursos e estratégias de incentivo ao uso das bibliotecas.

### Frequência pela CDU

In [ ]:
tabela_frequencia(todos_emprestimos, 'CDU')

De maneira geral, dá pra ver uma diferença bem grande entre os temas. A categoria Ciências Aplicadas domina com quase 69% dos empréstimos, o que mostra que o acervo e o interesse dos usuários estão muito voltados para áreas mais práticas e aplicadas, como saúde, engenharia e tecnologia. Em seguida, Ciências Sociais também tem um bom volume, com 17,8%, o que é esperado, já que muitas graduações e pesquisas orbitam essas áreas.

Por outro lado, temas como Geografia/Biografia/História, Belas Artes e Linguagem aparecem com números bem baixos (menos de 0,5% cada), o que chama atenção e levanta algumas questões:

- O acervo desses temas está atualizado ou está defasado?

- Esses temas atendem às necessidades do público atual da biblioteca?

- Está sendo feita alguma ação de divulgação ou incentivo para o uso desses materiais?

Pode ser que esses conteúdos estejam esquecidos, mal localizados ou pouco promovidos. Vale a pena a direção avaliar se é uma questão de falta de interesse do público ou de visibilidade do acervo. Uma ideia seria fazer campanhas temáticas ou destacar obras dessas áreas em períodos estratégicos, como durante eventos acadêmicos ou datas comemorativas. Também pode ser útil conversar com professores dessas áreas para entender se os alunos realmente têm procurado esse tipo de material ou se estão usando outras fontes.

### Análise Temporal dos Empréstimos com Boxplot

Agora vamos analisar a distribuição dos empréstimos ao longo do tempo com boxplots. A proposta é entender como o comportamento dos usuários da biblioteca como os alunos de graduação e pós-graduação varia de ano para ano, entre 2010 e 2020.

Para isso, selecionei a coleção com maior volume de empréstimos e criei uma função para gerar boxplots anuais, separados por tipo de usuário. Essa visualização permite identificar padrões, variações e possíveis outliers que ajudam a contar a história dos dados de forma clara.

Com essa análise, é possível levantar pontos importantes para a diretoria da biblioteca: há anos com maior ou menor volume de empréstimos? Algum comportamento atípico? A distribuição se manteve estável? Além disso, comparar graduação e pós-graduação traz uma visão mais detalhada sobre o perfil e as necessidades de cada grupo.

Esses insights contribuem diretamente para a avaliação e ajuste de estratégias de acervo e atendimento, considerando o histórico e as transformações ao longo dos anos.


In [ ]:
alunos_graduacao = todos_emprestimos.query('tipo_vinculo_usuario == "ALUNO DE GRADUAÇÃO"')
alunos_graduacao.colecao.value_counts()

### Frequência Mensal e por Ano de Empréstimos do Acervo Circulante por Alunos de Graduação

In [ ]:
alunos_graduacao_acervo_circulante = alunos_graduacao.query('colecao == "Acervo Circulante"').copy()

alunos_graduacao_acervo_circulante['data_emprestimo'] = pd.to_datetime(alunos_graduacao_acervo_circulante['data_emprestimo'])
alunos_graduacao_acervo_circulante['ano'] = alunos_graduacao_acervo_circulante['data_emprestimo'].dt.year
alunos_graduacao_acervo_circulante['mes'] = alunos_graduacao_acervo_circulante['data_emprestimo'].dt.month

resultado = (alunos_graduacao_acervo_circulante
             .groupby(['ano', 'mes'])
             .size()
             .reset_index(name='quantidade')
            .sort_values('quantidade', ascending=False))

resultado

In [ ]:
todos_emprestimos['data_emprestimo'] = pd.to_datetime(todos_emprestimos['data_emprestimo'])

### Função para gerar os boxplot

In [ ]:
def box_plot(dataset, x, y, titulo, subtitulo):
    '''
    Esta função gera um gráfico de boxplot com título e subtítulo customizados.
    
    :param dataset: DataFrame com os dados.
    :param x: Nome da variável categórica para o eixo X.
    :param y: Nome da variável numérica para o eixo Y.
    :param titulo: Título principal do gráfico.
    :param subtitulo: Subtítulo exibido abaixo do título.
    :return: Exibe o gráfico gerado.
    '''
    
    sns.set_theme(style="darkgrid", palette='Blues', font_scale=1.3)
    plt.figure(figsize=(16, 10))

    ax = sns.boxplot(x=x, y=y, data=dataset, color='#4171EF')

    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda val, _: f'{int(val):,}'.replace(',', '.')))
    plt.ylim(0, dataset[y].max() * 1.1)
    ax.set(xlabel=None, ylabel=None)
    ax.set_title(f"{titulo}\n", fontsize=20, loc='left', weight='bold')
    ax.text(x=-0.5, y=dataset[y].max() * 1.11, s=subtitulo, fontsize=18, ha='left', color='gray')

    plt.tight_layout()
    plt.show()

In [ ]:
box_plot(resultado,'ano','quantidade','Distribuição dos empréstimos mensais', 'Realizados pelos alunos de graduação no acervo circulante')

Esse recorte representa bem o panorama geral do conjunto de dados, já que estamos lidando com os alunos de graduação que é o grupo com maior volume de empréstimos e a coleção mais movimentada: o acervo circulante.

O boxplot nos ajuda a visualizar a distribuição mensal dos empréstimos ao longo dos anos. Fica evidente que houve uma queda a partir de 2013, com destaque para a forte baixa em 2019. Mas o que mais me chamou atenção foram os anos de 2014 e 2017.

Mesmo com outliers (provavelmente causados pelos meses de janeiro e dezembro), esses dois anos mostram uma distribuição mais estável dentro do período. Em 2014, há um leve declínio comparado a 2013, enquanto em 2017 há uma recuperação visível em relação a 2016.

Seria interessante entender com a direção das bibliotecas o que ocorreu nessas datas para que pudéssemos ter um contexto melhor da realidade do negócio.

### Aluno de pós graduação

In [ ]:
alunos_pos_graduacao = todos_emprestimos.query('tipo_vinculo_usuario == "ALUNO DE PÓS-GRADUAÇÃO"')
alunos_pos_graduacao.colecao.value_counts()

In [ ]:
alunos_pos_graduacao_acervo_circulante = alunos_pos_graduacao.query('colecao == "Acervo Circulante"').copy()

alunos_pos_graduacao_acervo_circulante['data_emprestimo'] = pd.to_datetime(alunos_pos_graduacao_acervo_circulante['data_emprestimo'])
alunos_pos_graduacao_acervo_circulante['ano'] = alunos_pos_graduacao_acervo_circulante['data_emprestimo'].dt.year
alunos_pos_graduacao_acervo_circulante['mes'] = alunos_pos_graduacao_acervo_circulante['data_emprestimo'].dt.month

resultado = (alunos_pos_graduacao_acervo_circulante
             .groupby(['ano', 'mes'])
             .size()
             .reset_index(name='quantidade')
            .sort_values('quantidade', ascending=False))

resultado

In [ ]:
box_plot(resultado,'ano','quantidade','Distribuição dos empréstimos mensais', 'Realizados pelos alunos de pós graduação no acervo circulante')

No caso dos alunos de pós-graduação, o comportamento dos empréstimos foi um pouco diferente quando comparamos com o grupo de graduação.

A partir de 2013, a linha mediana dos empréstimos mensais começou a cair de forma gradual até 2018, mas os valores máximos continuaram altos durante boa parte desse período, o que sugere certa estabilidade no volume total, mesmo com uma redução na consistência mensal. A queda mais acentuada só ocorreu de fato em 2019, o que indica uma possível mudança mais recente no comportamento dos usuários ou na oferta do serviço.

Assim como no gráfico da graduação, observamos um crescimento constante entre 2010 e 2013. Esse padrão nos leva a refletir sobre quais estratégias, políticas ou investimentos estavam sendo feitos nesse período que poderiam ter incentivado o uso das bibliotecas.

Nos dois casos, o ponto de atenção mais claro é o declínio acentuado a partir de 2018. Isso acende um alerta para que a direção das bibliotecas possa investigar o que mudou.

 Agora vamos analisar os empréstimos por curso de graduação, com foco nos dados entre 2015 e 2020. A proposta é atender a uma demanda comum nas universidades, que precisam fornecer indicadores que apoiem processos de avaliação externa, como os realizados pelo MEC. Os cursos selecionados são Biblioteconomia, Ciências sociais, Comunicação social, Direito, Filosofia e Pedagogia.

O objetivo é entender como se comporta o uso da biblioteca por parte dos cursos que serão avaliados, verificando o volume de empréstimos realizados ao longo dos anos. Para isso, foi necessário integrar dados de diferentes fontes, como planilhas em Excel e arquivos JSON da API do sistema, consolidando tudo em um único conjunto.

A partir dessa base unificada, criei uma tabela dinâmica que apresenta o total de empréstimos por curso e por ano, além de somatórios gerais por linha e por coluna. Isso permite visualizar com clareza quais cursos têm maior ou menor frequência de uso do acervo, como esse comportamento evoluiu no tempo e quais períodos tiveram mais movimentação.

Essas informações são fundamentais para a gestão da biblioteca, pois ajudam a demonstrar o impacto do serviço nos cursos de graduação e orientam decisões sobre políticas de aquisição, atendimento e suporte pedagógico.

### Coletando dados de arquivos EXCEL

In [ ]:
cadastro_alunos_antes_2010 = pd.read_excel('https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_6-Novos_dados_novas_analises/Datasets/matricula_alunos.xlsx',
                                        sheet_name='Até 2010',skiprows=1)
cadastro_alunos_apos_2010 = pd.read_excel('https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_6-Novos_dados_novas_analises/Datasets/matricula_alunos.xlsx',
                                        sheet_name='Após 2010',skiprows=1)

In [ ]:
cadastro_alunos_antes_2010

In [ ]:
cadastro_alunos_apos_2010

### Alterando o nome das colunas para a o mesmo formato dos dados anterioreres

In [ ]:
cadastro_alunos_antes_2010.columns = ['matricula_ou_siape','tipo_vinculo_usuario','curso']
cadastro_alunos_apos_2010.columns = ['matricula_ou_siape','tipo_vinculo_usuario','curso']

### Concatenando os arquivos EXCEL

In [ ]:
cadastro_alunos_excel = pd.concat([cadastro_alunos_antes_2010,cadastro_alunos_apos_2010],ignore_index=True)
cadastro_alunos_excel

In [ ]:
cadastro_alunos_excel.info()

### Modificando matricula_ou_siape para str

In [ ]:
cadastro_alunos_excel.matricula_ou_siape = cadastro_alunos_excel.matricula_ou_siape.astype(str)

### Coletando dados em JSON

In [ ]:
cadastro_alunos_json = pd.read_json('https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_6-Novos_dados_novas_analises/Datasets/cadastro_alunos.json')

In [ ]:
cadastro_alunos_json

In [ ]:
cadastro_alunos_graduacao_json = pd.read_json(StringIO(cadastro_alunos_json.registros[0]))

In [ ]:
cadastro_alunos_graduacao_json

In [ ]:
cadastro_alunos_graduacao_json.info()

### Modificando a matricula_ou_siape para str

In [ ]:
cadastro_alunos_graduacao_json.matricula_ou_siape = cadastro_alunos_graduacao_json.matricula_ou_siape.astype('float')
cadastro_alunos_graduacao_json.matricula_ou_siape = cadastro_alunos_graduacao_json.matricula_ou_siape.astype(str)

In [ ]:
cadastro_alunos_graduacao_json.info()

### Concatenando os arquivos 

In [ ]:
cadastro_alunos_graduacao = pd.concat([cadastro_alunos_excel,cadastro_alunos_graduacao_json],ignore_index=True)
cadastro_alunos_graduacao

### Empréstimos  entre 2015 e 2020

In [143]:
matricula_data_emprestimo = todos_emprestimos.query("tipo_vinculo_usuario == 'ALUNO DE GRADUAÇÃO'")
matricula_data_emprestimo = matricula_data_emprestimo.loc[:,['matricula_ou_siape','data_emprestimo']]
matricula_data_emprestimo = matricula_data_emprestimo.query('data_emprestimo >= 2015')
matricula_data_emprestimo = matricula_data_emprestimo.reset_index(drop=True)
matricula_data_emprestimo

,matricula_ou_siape,data_emprestimo
0,2013021576.0,2015-01-02 09:16:49.566
1,2014026648.0,2015-01-02 09:44:42.808
2,2014026648.0,2015-01-02 09:44:43.127
3,2010049586.0,2015-01-02 10:11:09.837
4,2014021454.0,2015-01-02 10:19:38.087
...,...,...
788099,20190154579.0,2020-03-18 16:26:20.477
788100,20160129109.0,2020-03-18 16:41:28.500
788101,20160129109.0,2020-03-18 16:41:28.588
788102,20160129109.0,2020-03-18 16:41:28.662


### Verificando valores ausentes

In [144]:
matricula_data_emprestimo.isnull().sum()

matricula_ou_siape    0
data_emprestimo       0
dtype: int64

In [145]:
print(cadastro_alunos_graduacao['curso'].unique())
print("Total:", len(cadastro_alunos_graduacao['curso'].unique()))

['MATEMÁTICA' 'CIÊNCIAS BIOLÓGICAS' 'LETRAS' 'ECOLOGIA'
 'ENGENHARIA AMBIENTAL' 'DIREITO' 'BIBLIOTECONOMIA'
 'CIÊNCIA DA COMPUTAÇÃO' 'SISTEMAS DE INFORMAÇÃO' 'CIÊNCIAS SOCIAIS'
 'ADMINISTRAÇÃO' 'ANÁLISE E DESENVOLVIMENTO DE SISTEMAS' 'NUTRIÇÃO'
 'COMUNICAÇÃO SOCIAL' 'GEOGRAFIA' 'ENGENHARIA DA COMPUTAÇÃO' 'TURISMO'
 'HISTÓRIA' 'ENGENHARIA QUÍMICA' 'DANÇA' 'MÚSICA' 'FISIOTERAPIA'
 'AGRONOMIA' 'FILOSOFIA' 'ENGENHARIA DE PETRÓLEO' 'PEDAGOGIA'
 'CIÊNCIAS E TECNOLOGIA' 'CIÊNCIAS ATUARIAIS' 'ENGENHARIA FLORESTAL'
 'ENFERMAGEM' 'ARQUITETURA E URBANISMO' 'EDUCAÇÃO FÍSICA' 'BIOMEDICINA'
 'ENGENHARIA CIVIL']
Total: 34


### Filtrando os cursos que serão analisados

In [146]:
cursos_selecionados = cadastro_alunos_graduacao.query("curso == ['BIBLIOTECONOMIA','CIÊNCIAS SOCIAIS','COMUNICAÇÃO SOCIAL','DIREITO','FILOSOFIA','PEDAGOGIA']")
cursos_selecionados


,matricula_ou_siape,tipo_vinculo_usuario,curso
8,200031011.0,ALUNO DE GRADUAÇÃO,DIREITO
9,200045080.0,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
15,200115053.0,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS
18,200123382.0,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
19,200123730.0,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
...,...,...,...
72789,20200097481.0,ALUNO DE GRADUAÇÃO,PEDAGOGIA
72796,20200107087.0,ALUNO DE GRADUAÇÃO,PEDAGOGIA
72797,20200012706.0,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
72800,20200009344.0,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA


### Unindo os Dataframes de cursos selecionados com matricula data emprestimo

In [147]:
cursos_selecionados = matricula_data_emprestimo.merge(cursos_selecionados)
cursos_selecionados['data_emprestimo'] = pd.to_datetime(cursos_selecionados['data_emprestimo'])
cursos_selecionados['data_emprestimo'] = cursos_selecionados.data_emprestimo.dt.year
cursos_selecionados

,matricula_ou_siape,data_emprestimo,tipo_vinculo_usuario,curso
0,2014021454.0,2015,ALUNO DE GRADUAÇÃO,FILOSOFIA
1,2013018187.0,2015,ALUNO DE GRADUAÇÃO,DIREITO
2,2014029499.0,2015,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS
3,2014029499.0,2015,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS
4,2014029499.0,2015,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS
...,...,...,...,...
212833,20200090284.0,2020,ALUNO DE GRADUAÇÃO,DIREITO
212834,20180113385.0,2020,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS
212835,20190150079.0,2020,ALUNO DE GRADUAÇÃO,PEDAGOGIA
212836,20190150079.0,2020,ALUNO DE GRADUAÇÃO,PEDAGOGIA


In [148]:
emprestimos_cursos_selecionados = cursos_selecionados.iloc[:,[1,3]].value_counts().reset_index()
emprestimos_cursos_selecionados.columns = ['ANO','CURSO','QUANTIDADE_EMPRESTIMOS']
emprestimos_cursos_selecionados

,ANO,CURSO,QUANTIDADE_EMPRESTIMOS
0,2017,COMUNICAÇÃO SOCIAL,8253
1,2016,COMUNICAÇÃO SOCIAL,7973
2,2017,DIREITO,7885
3,2015,BIBLIOTECONOMIA,7784
4,2015,DIREITO,7665
5,2016,FILOSOFIA,7576
6,2015,CIÊNCIAS SOCIAIS,7489
7,2017,PEDAGOGIA,7434
8,2017,FILOSOFIA,7389
9,2016,DIREITO,7363


In [149]:
emprestimos_aluno_curso_pivot = emprestimos_cursos_selecionados.pivot_table(
    index='CURSO',
    columns='ANO',
    values='QUANTIDADE_EMPRESTIMOS',
    fill_value=0,
    aggfunc='sum',
    margins=True,
    margins_name='TOTAL'
)
emprestimos_aluno_curso_pivot

ANO,2015,2016,2017,2018,2019,2020,TOTAL
CURSO,,,,,,,
BIBLIOTECONOMIA,7784,7044,7307,6378,5889,900,35302
CIÊNCIAS SOCIAIS,7489,6978,6918,6181,5788,825,34179
COMUNICAÇÃO SOCIAL,7147,7973,8253,6953,5885,796,37007
DIREITO,7665,7363,7885,7027,5861,905,36706
FILOSOFIA,6950,7576,7389,6296,5572,832,34615
PEDAGOGIA,7189,7153,7434,6518,5869,866,35029
TOTAL,44224,44087,45186,39353,34864,5124,212838
